# Display GEE remote sensing data on ESRI webmaps
This notebook provides a basic workflow to allow for the display of remote sensing images available on Google Earth Engine as layers on ESRI webmaps. The webmap and layers holding these images should be created and saved before running this script. The basic process is:

1. Images are first curated and processed on GEE.
2. We then define visualization parameters and generate tiles and publicly accessible urls. 
3. These are then used to update the tile url attributes of existing layers on an existing ESRI webmap. 

This notebook demonstrates the curation and addition of NAIP data, but the process can be used for any image assets available on GEE, including NLCD, Sentinel-2, Sentinel-1, and SRTM Digital Elevation data.

## Setup

In [3]:
# import arc libraries
from arcgis.gis import GIS
from arcgis.mapping import WebMap

import pandas as pd

In [4]:
# import and authenticate gee library
import ee

# trigger an interactive session that will allow you to authenticate to an existing GEE account through a browser
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [45]:
# define some global variables
USER = 'jczawlytko' # your ArcOnline account that can access organizational assets
PASSWORD = '!!Oyster2018' # password for above account
WEBMAP = 'NAIP_tile_generator' # title of the existing webmap to be updated

## GEE
Here we use Google Earth Engine to generate new tile urls for naip imagery

In [12]:
# get NAIP and TIGER catalogs
NAIP = ee.ImageCollection("USDA/NAIP/DOQQ")
TIGER = ee.FeatureCollection("TIGER/2018/States")


In [13]:
NAIP_datesURL = r"https://raw.githubusercontent.com/conservation-innovation-center/landuse/develop/lookup_tables/landcover_dates.csv?token=GHSAT0AAAAAAB535E7NARE243YXZUWK7RQCY6QJ5MQ"
df = pd.read_csv(NAIP_datesURL,index_col=0)
# df = pd.read_csv(NAIP_dates)
print(df.columns)

Index(['State', 'T1', 'T2', 'co_fips'], dtype='object')


In [40]:
df.head()

,State,T1,T2,co_fips
County,,,,
District of Columbia,DC,2013,2017,wash_11001
Kent,DE,2013,2018,kent_10001
New Castle,DE,2013,2018,newc_10003
Sussex,DE,2013,2018,suss_10005
Allegany,MD,2013,2018,alle_24001


In [8]:
# define image visualization parameters
naip_viz_params = {
  'bands':['R', 'G', 'B'],
  'min': 0,
  'max': 200
}

In [51]:
# create a dictionary to hold tiles by state keyword
TILES = {}

# loop thru df

for index, row in df.iterrows():
    # get state name and T1 and T2 NAIP years
    State = row ['State']
    T1 = row['T1']
    T2 = row['T2']
    print(f"Generating tiles for {row ['State']}                ", end='\r')
    
    # get stat bounds
    st_bounds = TIGER.filterMetadata('NAME', 'equals', State)

    #create state naip collection then filter by T1 and T2 dates
    st_naip = NAIP.filterBounds(st_bounds)
    naipT1 = st_naip.filterDate(f'{T1}-01-01', f'{T1}-12-31').median()
    naipT2 = st_naip.filterDate(f'{T2}-01-01', f'{T2}-12-31').median()

    #create 
    map_id_dict1 = ee.Image(naipT1).getMapId(naip_viz_params)
    map_id_dict2 = ee.Image(naipT2).getMapId(naip_viz_params)
    tiles1 = map_id_dict1['tile_fetcher'].url_format
    tiles2 = map_id_dict2['tile_fetcher'].url_format
    TILES[State] = [tiles1, tiles2]
    
print(f"{len(TILES)} sets of tiles generated                        ")

7 sets of tiles generated                        


In [52]:
TILES

{'DC': ['https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/maps/394619e3ab182886ae5e8c39d09c4630-cbe04548edc7c1e6aac9fbf0bc8e4e51/tiles/{z}/{x}/{y}',
  'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/maps/a0e879da31c8fd512e8b2247092ef755-4a8bb95f84152637b0084189f768cdde/tiles/{z}/{x}/{y}'],
 'DE': ['https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/maps/394619e3ab182886ae5e8c39d09c4630-77954d57eed0a25a177fb62c7c815e11/tiles/{z}/{x}/{y}',
  'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/maps/3ce5c8e1b7c13951a0ca86a1124b5b99-404f42ba889f0a4fbc4b191504bde116/tiles/{z}/{x}/{y}'],
 'MD': ['https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/maps/394619e3ab182886ae5e8c39d09c4630-15bfddfb9193323374ec17bfefe22638/tiles/{z}/{x}/{y}',
  'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/maps/3ce5c8e1b7c13951a0ca86a1124b5b99-89d6cf0006f4004d902792b7cc85e921/tiles/{z}/

## ESRI webmap
In this section we update the tile urls attributes of the layers on an ESRI webmap. This webmap and layers should exist already  

In [53]:
# first we create a arconline client with our credentials

gis = GIS(username =  USER, password = PASSWORD)

In [46]:
# find the instance of the webmap we want to display gee imagery on
search_results = gis.content.search(query = f'title:{WEBMAP}', item_type = 'Web Map')
wm_item = search_results[0]

In [47]:
# cast the existing webmap into a WebMap class object
web_map_obj = WebMap(wm_item)

In [54]:
# obtain the layers that will display NAIP data
layers = web_map_obj.layers
naip_layers = [layer for layer in layers if 'NAIP' in layer['title']]

[{
   "templateUrl": "https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/maps/394619e3ab182886ae5e8c39d09c4630-2eed49ecf7d74539b7d2c8bfe97d3994/tiles/{z}/{x}/{y}",
   "copyright": "Google Earth Engine, USGS",
   "fullExtent": {
     "xmin": -20037508.342787,
     "ymin": -20037508.34278,
     "xmax": 20037508.34278,
     "ymax": 20037508.342787,
     "spatialReference": {
       "wkid": 102100
     }
   },
   "opacity": 1,
   "visibility": true,
   "id": "WebTiled_6130",
   "title": "NAIP DC 2017",
   "type": "WebTiledLayer",
   "layerType": "WebTiledLayer"
 },
 {
   "templateUrl": "https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/maps/a0e879da31c8fd512e8b2247092ef755-ec43e170592b7bf7fa88dceb6ef52c23/tiles/{z}/{x}/{y}",
   "copyright": "Google Earth Engine, USGS",
   "fullExtent": {
     "xmin": -20037508.342787,
     "ymin": -20037508.34278,
     "xmax": 20037508.34278,
     "ymax": 20037508.342787,
     "spatialReference": {
       "wkid": 10

In [55]:
print(len(TILES.items()))
for state, tiles in TILES.items():
    print(state,tiles)
    print()
    layers = [naip_layer for naip_layer in naip_layers if state in naip_layer['title']]
    [layers[i].update({'templateUrl':tile}) for i, tile in enumerate( tiles)]

7
DC ['https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/maps/394619e3ab182886ae5e8c39d09c4630-cbe04548edc7c1e6aac9fbf0bc8e4e51/tiles/{z}/{x}/{y}', 'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/maps/a0e879da31c8fd512e8b2247092ef755-4a8bb95f84152637b0084189f768cdde/tiles/{z}/{x}/{y}']

DE ['https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/maps/394619e3ab182886ae5e8c39d09c4630-77954d57eed0a25a177fb62c7c815e11/tiles/{z}/{x}/{y}', 'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/maps/3ce5c8e1b7c13951a0ca86a1124b5b99-404f42ba889f0a4fbc4b191504bde116/tiles/{z}/{x}/{y}']



IndexError: list index out of range

In [59]:
naip_layers

[{
   "templateUrl": "https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/maps/394619e3ab182886ae5e8c39d09c4630-cbe04548edc7c1e6aac9fbf0bc8e4e51/tiles/{z}/{x}/{y}",
   "copyright": "Google Earth Engine, USGS",
   "fullExtent": {
     "xmin": -20037508.342787,
     "ymin": -20037508.34278,
     "xmax": 20037508.34278,
     "ymax": 20037508.342787,
     "spatialReference": {
       "wkid": 102100
     }
   },
   "opacity": 1,
   "visibility": true,
   "id": "WebTiled_6130",
   "title": "NAIP DC 2017",
   "type": "WebTiledLayer",
   "layerType": "WebTiledLayer"
 },
 {
   "templateUrl": "https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/maps/a0e879da31c8fd512e8b2247092ef755-4a8bb95f84152637b0084189f768cdde/tiles/{z}/{x}/{y}",
   "copyright": "Google Earth Engine, USGS",
   "fullExtent": {
     "xmin": -20037508.342787,
     "ymin": -20037508.34278,
     "xmax": 20037508.34278,
     "ymax": 20037508.342787,
     "spatialReference": {
       "wkid": 10

In [60]:
web_map_obj.update()

True